# 電力交易市場數據分析課程 2024/05/31

In [ ]:
import pandas as pd
import numpy as np

## 電力週報案例一：市場價格趨勢

In [ ]:
# colab
url = 'https://raw.githubusercontent.com/godspeed833/course_demo_files/master/vpp_tbl_etp_historical_settlement_trading_colab.csv'
df_etp = pd.read_csv(url)

In [ ]:
df_etp

In [ ]:
df_etp['datetime'] = df_etp['tranDate'] + ' ' + df_etp['tranHour']
df_etp['datetime'] = pd.to_datetime(df_etp['datetime'])
df_etp['month'] = df_etp.datetime.dt.to_period('M').astype(str)

In [ ]:
df_etp.columns

In [ ]:
#df_etp2 = df_etp[df_etp['datetime'] > '2022-01-01'][['datetime', 'regBidQse', 'regPrice', 'srBidQse', 'srPrice', 'supBidQse', 'supPrice']]
df_etp2 = df_etp[df_etp['datetime'] > '2022-01-01'][['datetime', 'month', 'regPrice', 'srPrice', 'supPrice']]

In [ ]:
df_etp2.head()

In [ ]:
df_etp2.regPrice

In [ ]:
df_etp2.regPrice.describe()

In [ ]:
df_etp2.info()

In [ ]:
df_etp2.isnull().sum()

In [ ]:
# 缺值視沒有交易，所以不補值
df_etp2[df_etp2.srPrice.isnull()]

In [ ]:
df_etp2.groupby(df_etp2.datetime.dt.to_period('M')).agg({'regPrice':'mean'})

In [ ]:
df_etp2.groupby([df_etp2.month, df_etp2.datetime.dt.hour]).agg({'regPrice':'mean','srPrice':'mean','supPrice':'mean'})

In [ ]:
df_etp3 = df_etp2.groupby([df_etp2.month,df_etp2.datetime.dt.hour]).agg({'regPrice':'mean','srPrice':'mean','supPrice':'mean',}).reset_index()
df_etp3.round().head()

In [ ]:
# pivot_table: 一個值，以row跟columns的方式展開
df_etp3.pivot_table(index='month', columns='datetime', values='regPrice')

In [ ]:
import plotly.graph_objects as go

In [ ]:
# Create traces
fig = go.Figure()

for month in df_etp3.month.unique():
    tmp = df_etp3[df_etp3.month == month]
    fig.add_trace(go.Scatter(x=tmp.datetime, y=tmp.regPrice,
                    mode='lines',
                    name=month))

fig.show()

In [ ]:
import matplotlib.pyplot as plt

# 創建時間順序的顏色漸變並反轉顏色比例
unique_months = df_etp3['month'].unique()
color_scale = plt.cm.viridis(np.linspace(0, 1, len(unique_months)))

# 創建具有顏色漸變的 traces
fig = go.Figure()

for idx, month in enumerate(unique_months):
    tmp = df_etp3[df_etp3['month'] == month]
    color = f'rgba({color_scale[idx][0]*255},{color_scale[idx][1]*255},{color_scale[idx][2]*255},1)'
    fig.add_trace(go.Scatter(x=tmp['datetime'], y=tmp['regPrice'],
                             mode='lines',
                             name=month,
                             line=dict(color=color)))

fig.update_layout(
    title="調頻價格隨時間的變化",
    xaxis_title="24小時",
    yaxis_title="調頻價格",
    legend_title="月份"
)

fig.show()


In [ ]:
import plotly.express as px
import pandas as pd

# 假设 df_etp3 包含了你的数据
# 可以透過类似以下的方式转换数据，以适应 Plotly Express 的格式要求
# df = pd.DataFrame({
#     'datetime': df_etp3['datetime'],
#     'regPrice': df_etp3['regPrice'],
#     'month': df_etp3['month']
# })

# 创建动画
fig = px.line(df_etp3, x='datetime', y='regPrice', animation_frame='month',
              title='Monthly regPrice Change', range_y=[0, 600])

# 显示动画
fig.show()


### 我將上面的程式碼貼給Chatgpt，問他如何產生dash介面
![image.png](attachment:image.png)

In [ ]:
!pip install jupyter-dash -q

In [ ]:
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


# 创建 Dash 应用
# app = dash.Dash(__name__)
app = JupyterDash(__name__)

# 布局
app.layout = html.Div([
    dcc.Checklist(
        id='price-selector',
        options=[
            {'label': 'regPrice', 'value': 'regPrice'},
            {'label': 'suPrice', 'value': 'srPrice'},
            {'label': 'spPrice', 'value': 'supPrice'}
        ],
        value=['regPrice'],  # 默认选中 'regPrice'
    ),
    dcc.Graph(id='price-graph')
])

# 回调函数，更新图表
@app.callback(
    Output('price-graph', 'figure'),
    [Input('price-selector', 'value')]
)
def update_graph(selected_prices):
    # 创建动画
    fig = px.line(df_etp3, x='datetime', y=selected_prices, animation_frame='month',
                  title='Monthly Price Change', range_y=[0, 600])
    return fig

# 在colab運行時，不需要指定mode，不然會跑出兩個
app.run_server(port=8035)

## 電力週報案例二 機組發電資料分析

In [ ]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

# 定義數據
data = {
    'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00'],
    '太陽能': [23.7, 23.7, 23.7, 23.7, 23.7, 23.7, 23.7, 23.7],
    '其他再生能源': [2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1],
    '儲能': [0, 0, 0, 0, 0, 0, 0, 0],
    '風力': [23.5, 23.5, 23.5, 23.5, 23.5, 23.5, 23.5, 23.5],
    '水力': [20.2, 20.2, 20.2, 20.2, 20.2, 20.2, 20.2, 20.2],
    '抽蓄': [3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6],
    '燃油': [16.2, 16.2, 16.2, 16.2, 16.2, 16.2, 16.2, 16.2],
    '民營電廠-燃氣': [225.6, 225.6, 225.6, 225.6, 225.6, 225.6, 225.6, 225.6],
    '燃氣': [109.5, 109.5, 109.5, 109.5, 109.5, 109.5, 109.5, 109.5],
    '燃煤': [964.9, 964.9, 964.9, 964.9, 964.9, 964.9, 964.9, 964.9],
    '民營電廠-燃煤': [280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0],
    '核能': [792.2, 792.2, 792.2, 792.2, 792.2, 792.2, 792.2, 792.2],
    '燃煤': [188.0, 188.0, 188.0, 188.0, 188.0, 188.0, 188.0, 188.0]
}

# 轉換為 DataFrame
df = pd.DataFrame(data)

# 創建堆積面積圖
fig = go.Figure()

for col in df.columns[1:]:
    fig.add_trace(go.Scatter(
        x=df['time'],
        y=df[col],
        mode='lines',
        stackgroup='one',
        name=col
    ))

# 設置圖表標題和標籤
fig.update_layout(
    title='今日用電曲線圖 - 依燃料類別 (Load Curve of Today - by Fuel Type)',
    xaxis_title='時間',
    yaxis_title='單位: 百萬瓦',
    yaxis=dict(type='linear'),
    legend_title='燃料類別'
)

# 顯示圖表
fig.show()


In [ ]:
import plotly.graph_objs as go
import pandas as pd

# 定義數據
data = {
    'time': ['00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00'],
    '太陽能': [23.7, 23.7, 23.7, 23.7, 23.7, 23.7, 23.7, 23.7],
    '其他再生能源': [2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.1],
    '儲能': [0, 0, 0, 0, 0, 0, 0, 0],
    '風力': [23.5, 23.5, 23.5, 23.5, 23.5, 23.5, 23.5, 23.5],
    '水力': [20.2, 20.2, 20.2, 20.2, 20.2, 20.2, 20.2, 20.2],
    '抽蓄': [3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6, 3.6],
    '燃油': [16.2, 16.2, 16.2, 16.2, 16.2, 16.2, 16.2, 16.2],
    '民營電廠-燃氣': [225.6, 225.6, 225.6, 225.6, 225.6, 225.6, 225.6, 225.6],
    '燃氣': [109.5, 109.5, 109.5, 109.5, 109.5, 109.5, 109.5, 109.5],
    '燃煤': [964.9, 964.9, 964.9, 964.9, 964.9, 964.9, 964.9, 964.9],
    '民營電廠-燃煤': [280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0, 280.0],
    '核能': [792.2, 792.2, 792.2, 792.2, 792.2, 792.2, 792.2, 792.2],
    '燃煤2': [188.0, 188.0, 188.0, 188.0, 188.0, 188.0, 188.0, 188.0]
}

# 轉換為 DataFrame
df = pd.DataFrame(data)

# 創建堆積面積圖
fig = go.Figure()

for col in df.columns[1:]:
    fig.add_trace(go.Scatter(
        x=df['time'],
        y=df[col],
        mode='lines',
        stackgroup='one',
        name=col,
        hoverinfo='x+y+name'
    ))

# 設置圖表標題和標籤
fig.update_layout(
    title='今日用電曲線圖 - 依燃料類別 (Load Curve of Today - by Fuel Type)',
    xaxis_title='時間',
    yaxis_title='單位: 百萬瓦',
    yaxis=dict(type='linear'),
    legend_title='燃料類別',
    hovermode='x unified'  # 統一顯示 hover 信息
)

# 顯示圖表
fig.show()


## 電力週報案例二：頻率觸發時間分析

### 向量化與傳統for迴圈的比較

In [ ]:
a = [1, 2, 3, 4, 5]
b = [1, 2, 3, 4, 5]
[x * y for x, y in zip(a, b)]


In [ ]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([1, 2, 3, 4, 5])
a * b

In [ ]:
import random
a = [random.randint(1, 10) for _ in range(1000000)]
b = [random.randint(1, 10) for _ in range(1000000)]
%timeit res = [x * y for x, y in zip(a, b)]

In [ ]:
a = np.random.randint(1, 10, 1000000)
b = np.random.randint(1, 10, 1000000)
%timeit a * b

### 頻率分析

In [ ]:
# Local Site only
url = 'https://raw.githubusercontent.com/godspeed833/course_demo_files/master/tpc_freq_colab.csv'
df_tpc = pd.read_csv(url)


In [ ]:
df_tpc.timestamp = pd.to_datetime(df_tpc.timestamp)

In [ ]:
df_tpc

In [ ]:
df_tpc.shape

In [ ]:
# def freq_sum(value):
#     return value.count()/10627200
# df_tpc.pivot_table('tpc', [], bin_freq, aggfunc=[freq_sum] )

In [ ]:
bin_freq = pd.cut(df_tpc['tpc'], [59, 59.75, 59.86, 59.98, 60, 60.02, 60.14, 60.25, 61 ])
df_tpc.pivot_table('tpc', df_tpc.timestamp.dt.month, bin_freq, aggfunc=['count'] )


In [ ]:
pd.cut(df_tpc['tpc'], [59, 59.75, 59.86, 59.98, 60, 60.02, 60.14, 60.25, 61 ])

In [ ]:
def get_sReg_fast(freq):
    sreg_trigger = np.where(freq <= 59.88, 1, np.nan)
    sreg_trigger = np.where(freq >= 59.98, 0, sreg_trigger)
    if pd.isna(sreg_trigger[0]):    # 給起始一個值
        sreg_trigger[0] = 0
    sreg_trigger = pd.Series(sreg_trigger).fillna(method="ffill")

    return(sreg_trigger)

In [ ]:
def get_sReg(freq):
    sreg_trigger = pd.Series([0]*len(freq))
    flag = False
    for i, value in enumerate(freq):
        if value <= 59.88: #觸發條件
            flag = True
        elif (value >= 59.98): #結束條件
            flag = False
        if flag:
            sreg_trigger[i] = 1

    return(sreg_trigger)

In [ ]:
# 只比前100天的頻率資料
df_tpc2 = df_tpc.iloc[:86400*100, :].copy()

In [ ]:
# 1. 加速版本
a = get_sReg_fast(df_tpc2.tpc)

In [ ]:
# 2. 原始版本
b = get_sReg(df_tpc2.tpc)

In [ ]:
(a == b).sum()

In [ ]:
get_sReg_fast(df_tpc2.tpc).value_counts()

In [ ]:
get_sReg(df_tpc2.tpc).value_counts()

In [ ]:
def get_sReg_chatgpt(freq):
    sreg_trigger = (freq <= 59.88).astype(int)
    flag_mask = (freq >= 59.98) & sreg_trigger.astype(bool)
    sreg_trigger[flag_mask.shift().fillna(False)] = 0
    return sreg_trigger


In [ ]:
get_sReg_chatgpt(df_tpc2.tpc).value_counts()

### 將標記完資料，統計持續時間及執行次數

In [ ]:
!pip install pyrle -q

In [ ]:
from pyrle import Rle

In [ ]:
# 計算1連續的長度
def gen_rle_length(series):
    a = Rle(series)
    sReg_sec = [x for x, y in zip(a.runs, a.values) if y == 1]
    return sReg_sec

# 產生一個array1的value的起始或結束位置的index
def gen_endpoint_index(array1, isStart, value):
    if isStart == True:
        index1 = [i for i in range(len(array1)) if array1[i] == value and (i == 0 or array1[i - 1] != value)]
    else:
        index1 = [i for i in range(len(array1)) if array1[i] == value and (i == (len(array1)-1) or array1[i + 1] != value)]

    return index1


In [ ]:
df_tpc2['sreg_triggered'] =  get_sReg_fast(df_tpc2.tpc)

In [ ]:
df_tpc2.sreg_triggered.value_counts()

In [ ]:
df_tpc2.head()

In [ ]:
sreg_len = gen_rle_length(df_tpc2.sreg_triggered)
start_idx_lst = gen_endpoint_index(df_tpc2.sreg_triggered, True, 1.0)


In [ ]:
df_result = df_tpc2.loc[start_idx_lst].copy()
df_result['time_len'] = sreg_len
#df_result[['timestamp', 'time_len']]

In [ ]:
df_result

In [ ]:
df_result.groupby(df_result.timestamp.dt.to_period('W')).agg({'sreg_triggered':'count', 'time_len':'mean'}).round()

In [ ]:
df_result.info()


In [ ]:
df_result.timestamp.dt.to_period('W')

## 電力週報案例三 用電大戶分析

In [ ]:
type = 4
# 高壓三段式
if type == 3:
    df_tou = pd.read_csv('https://raw.githubusercontent.com/godspeed833/course_demo_files/master/tou_price_%E9%AB%98%E5%A3%93%E4%B8%89%E6%AE%B5%E5%BC%8F_2023.csv')
# 高壓批次生產
elif type == 4:
    df_tou = pd.read_csv('https://raw.githubusercontent.com/godspeed833/course_demo_files/master/tou_price_%E9%AB%98%E5%A3%93%E6%89%B9%E6%AC%A1_2023.csv')

df_tou.datetime = pd.to_datetime(df_tou.datetime)
df_tou

In [ ]:
# 平均電價
df_tou.tou.mean()

In [ ]:
df_tou2 = df_tou.groupby(df_tou.datetime.dt.hour).tou_tag.value_counts().reset_index(name='count')
df_tou2

In [ ]:
tou_price = df_tou.groupby(df_tou.datetime.dt.hour).tou.mean()

In [ ]:


# use stack bar plot to show the distribution of tou of each hour
fig = go.Figure()

# 修正stack bar plot的次序，非夏月_離峰,非夏月_週六半尖峰, 非夏月_尖峰,夏月_離峰,夏月_半尖峰, 夏月_週六半尖峰,夏月_尖峰
if type == 3:
# 三段式
    tou_tag_lst = ['非夏月_離峰', '非夏月_週六半尖峰', '非夏月_尖峰', '夏月_離峰', '夏月_週六半尖峰', '夏月_半尖峰' , '夏月_尖峰']
elif type == 4:
# 批次
    tou_tag_lst = ['非夏月_離峰', '非夏月_週六半尖峰', '非夏月_尖峰', '夏月_離峰', '夏月_週六半尖峰', '夏月_尖峰']

df_tou2['tou_tag'] = pd.Categorical(df_tou2['tou_tag'], categories=tou_tag_lst, ordered=True)

df_tou2 = df_tou2.sort_values(by=['datetime', 'tou_tag'])
df_tou2['tou_tag'] = df_tou2['tou_tag'].astype(str)

# fig = go.Figure()
for tou_tag in tou_tag_lst:
    # 選擇 非夏月_離峰,非夏月_週六半尖峰, 非夏月_尖峰,夏月_離峰,夏月_半尖峰, 夏月_週六半尖峰,夏月_尖峰的顏色，
    # 給個紅色漸層，由淺到深，夏月_尖峰最深

    # 三段式
    if type == 3:
        if tou_tag == '非夏月_離峰':
            color = 'rgb(255, 255, 255)'
        elif tou_tag == '非夏月_週六半尖峰':
            color = 'rgb(255, 214, 214)'
        elif tou_tag == '非夏月_尖峰':
            color = 'rgb(255, 163, 163)'
        elif tou_tag == '夏月_離峰':
            color = 'rgb(255, 112, 112)'
        elif tou_tag == '夏月_週六半尖峰':
            color = 'rgb(255, 61, 61)'
        elif tou_tag == '夏月_半尖峰':
            color = 'rgb(200, 0, 0)'
        elif tou_tag == '夏月_尖峰':
            color = 'rgb(139, 0, 0)'
    elif type == 4:
        # 批次
        if tou_tag == '非夏月_離峰':
            color = 'rgb(255, 255, 255)'
        elif tou_tag == '非夏月_週六半尖峰':
            color = 'rgb(255, 214, 214)'
        elif tou_tag == '非夏月_尖峰':
            color = 'rgb(255, 163, 163)'
        elif tou_tag == '夏月_離峰':
            color = 'rgb(255, 112, 112)'
        elif tou_tag == '夏月_週六半尖峰':
            color = 'rgb(200, 0, 0)'
        elif tou_tag == '夏月_尖峰':
            color = 'rgb(139, 0, 0)'

    # Add a tou_price line on the plot, 使用不同的y軸
    fig.add_trace(go.Bar(x=df_tou2[df_tou2.tou_tag == tou_tag].datetime, y=df_tou2[df_tou2.tou_tag == tou_tag]['count'], name=tou_tag, marker_color=color))

# tou_price line使用亮一點的藍色跟粗一點的線
fig.add_trace(go.Scatter( y=tou_price, name='tou_price', yaxis='y2', line=dict(color='blue', width=2)))
# fig.add_trace(go.Scatter( y=tou_price, name='tou_price', yaxis='y2', line=dict(color='blue', width=1)))
# lengend 的位置放在下方，只保留barplot的lengend，tou_price的lengend不顯示
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

if type == 3:
    fig.update_layout(barmode='stack', xaxis_title='Hour', yaxis_title='小時數', title='2023年的高壓三段式時間電價分佈圖')
elif type == 4:
    fig.update_layout(barmode='stack', xaxis_title='Hour', yaxis_title='0.5小時數', title='2023年的高壓批次生產時間電價分佈圖')


fig.update_layout(yaxis2=dict(title='小時平均價格', overlaying='y', side='right'))



fig.show()

    # fig.add_trace(go.Bar(x=df_tou2[df_tou2.tou_tag == tou_tag].datetime, y=df_tou2[df_tou2.tou_tag == tou_tag]['count'], name=tou_tag, marker_color=color))




# 到此結束，謝謝大家